# Homework: Sudoku puzzle solver --- backtracking

In this exercise, you are asked to write a backtracking program to solve a partially-filled Sudoku puzzle.

The fully filled sudoku grid, is a $9\times 9$ square of numbers, where each digits from $1$ to $9$ appears exactly one in each column, each row, and each $3 \times 3$ box. A Sudoku puzzle is a partially filled $9 \times 9$ grid like, with digits in some of the cells, and empty cells otherwise. The task is to fill in all missing digits, such that the final grid is valid in the sense above.

You can see  some of the Sudoku puzzles (together with how a grid is partitioned into $3 \times 3$ boxes) at [Web Sudoku](https://www.websudoku.com/?level=1) service. 

We will represent a (partially-filled) sudoku grid as a list containing exactly $9$ lists, each of which is a list with $9$ elements --- the elements are going to be digit from $0$ to $9$, with $0$ signifying an empty cell. For convenience below are two partially filled sudoku grids from one of the online web services:

In [86]:
sudoku = [[5,6,0,9,0,0,4,2,7],
          [3,0,0,6,8,0,9,0,0],
          [0,9,0,0,0,4,0,0,0],
          [0,2,0,0,0,5,8,0,1],
          [8,0,1,0,2,0,3,0,0],
          [4,0,0,8,9,0,0,5,6],
          [7,3,0,1,0,0,0,9,0],
          [2,1,9,0,0,0,0,6,4],
          [0,8,0,4,0,0,0,3,2]]

In [87]:
sudoku_extreme = [[0,0,0,9,0,0,0,0,7],
                  [0,0,0,8,0,1,0,9,0],
                  [8,0,1,4,0,0,0,0,0],
                  [9,0,0,5,0,8,4,0,0],
                  [0,0,4,0,0,0,0,3,0],
                  [0,0,7,0,0,0,9,0,0],
                  [0,0,0,6,0,0,0,0,3],
                  [1,0,2,0,3,4,0,0,0],
                  [5,0,0,0,0,0,0,0,0]]

You are asked to write a function `solve_sudoku(sudoku)` that takes a partially filled sudoku (puzzle), and returns a fully filled sudoku --- i.e. fills all the empty cells in `sudoku` in a way consistent with the sudoku rules - each row, column and box contains all the digits from $1$ to $9$, once each.

In this homework I will provide you already with an implementation of several functions that are going to be useful. Each of those is by itself relatively simple, and if you want to practice more, try to write them yourself one by one, reading only the description of what the function is supposed to do (and debug it right after writing).

For a less time-consuming variant of this homework, read through the code of all functions below and the description of the algorithm, and try only implementing the main part of the algorithm, using the functions provided here.

## Get all indices from box

**Exercise 1**
The first primitive that will be useful, is a function `get_all_indices_from_box` that takes as an argument a position of a digit `pos` --- a tuple of two numbers, each between $0$ to $8$ indicating the row and the column of the position we care about --- and produces a list of all positions of digits in the same box as `pos`.

For example

```python
get_all_indices_from_box((1,1)) == [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
get_all_indices_from_box((3,0)) == [(3, 0), (3, 1), (3, 2), (4, 0), (4, 1), (4, 2), (5, 0), (5, 1), (5, 2)]
```

**Solution**

In [70]:
def get_all_indices_from_box(pos):
    box_row = (pos[0]//3)
    box_column = (pos[1]//3)
    return [(box_row * 3 + i, box_column*3 + j) for i in range(3) for j in range(3)]

In [71]:
get_all_indices_from_box((3,0))

[(3, 0), (3, 1), (3, 2), (4, 0), (4, 1), (4, 2), (5, 0), (5, 1), (5, 2)]

In [57]:
ALL_DIGITS = [i+1 for i in range(9)]

## Finding available digits

We would like to write a function that takes as an argument a partially-filled sudoku grid `partial_sudoku` as well as position `pos` of some empty cell, and produces a list of all digits that can be written into the empty cell without directly violating any constraint. That is, we want to produce a list of all digits from $1$ to $9$ that do not appear so far in the same row, column or box as `pos` in the `partial_sudoku`.

### Exercise
As a first step, write a simple function `is_in_any` that takes as an argument `target` and a list `lists` of list. The function should return `True` if `x` is an element of any of the lists `lists`.

**Example**
```python
is_in_any(3, [[2,5,6], [1,3,6], [1,2,4]]) == True
```
since $3$ is an element of the second list `[1,3,6]`. On the other hand
```python
is_in_any(7, [[2,5,6], [1,3,6], [1,2,4]]) == False
```
since $7$ does not appear in any of those lists.

**Solution**

In [74]:
def is_in_any(target, lists):
    for lst in lists:
        if target in lst:
            return True
    return False

In [75]:
is_in_any(3, [[2,5,6], [1,3,6], [1,2,4]])

True

In [76]:
is_in_any(7, [[2,5,6], [1,3,6], [1,2,4]])

False

### Exercise
Write a function `get_available_digits` that takes two arguments: `partial_sudoku` --- a partially filled sudoku grid as discussed above, and `pos` --- a pair of two numbers between `0` and `8` indicating the row and the column of the cell under consideration. The function should return the list of digits that can be inserted in this cell.

To this end, produce three lists: one containing all digits from the same row as `pos` in the sudoku `partial_sudoku`, the other containing all digits from the same column, and finally the third containing all digits from the same box as `pos`.

Now iterate over all digits from $1$ to $9$ and use the function `is_in_any` written before, to check if the digit appears in any of the lists of non-available digits. If it does not --- add it to the list of available digits.

**Hint**
You can use python list comprehension syntax to make the code shorter and cleaner.

**Example**
For the partial sudoku `sudoku` defined at the beginning, we have
```python
get_available_digits(sudoku, (0, 4)) == [1,3]
```
the digits $2,4,5,6,7,9$ are excluded because they appear in the row $0$. The digit $8$ is excluded because it appears in the row $4$ (and in the same box). The ramaining digits are $1$ and $3$.

**Solution**

In [83]:
def get_available_digits(partial_sudoku, pos):
    if partial_sudoku[pos[0]][pos[1]] != 0:
        return [partial_sudoku[pos[0]][pos[1]]]
 
    row_digits = partial_sudoku[pos[0]]
    column_digits = [ partial_sudoku[j][pos[1]] for j in range(9) ]
    box_digits = [partial_sudoku[x][y] for (x,y) in get_all_indices_from_box(pos)]       
    
    forbidden_digits = [column_digits, row_digits, box_digits]
    return [x for x in range(1, 10) if not is_in_any(x, forbidden_digits)]

In [84]:
get_available_digits(sudoku, (0, 4))

[1, 3]

**Exercise**
Final short function to write is a `next_pos` that takes as an input position `pos` -- a pair of two numbers, and returns the next position of a digit to consider, in a standard "reading order": if `pos` contain to the last cell in a row, we should move to the first cell in the next row, otherwise we should move to the next cell in the same row as `pos`.

**Solution**

In [85]:
def next_pos(pos):
    if pos[1] < 8:
        return (pos[0], pos[1] + 1)
    else:
        return (pos[0] + 1, 0)

### Recursive Backtracking Algorithm for Solving Sudoku --- the main problem

Your final task is to implement a recursive, backtracking-based function that solves a partially filled Sudoku grid. Conceptually, this algorithm is very similar to the one we discussed for the $n$-queens problem.

The function, `solve_sudoku`, should take two arguments:
- `sudoku` — a partially filled Sudoku grid
- `pos` — the current position (cell) to consider
  
You may assume that all cells before `pos` are already filled with valid digits. Some cells at or after `pos` may still be empty. The goal of the function is to fill in the remaining cells so that the grid becomes a complete, valid Sudoku solution.

---
**Algorithm Outline**
1. **Base Case:**
If `pos` refers to a location beyond the last cell of the grid, it means we have successfully filled the entire Sudoku. In this case, simply return the completed grid as the solution.
2. **Skip Pre-Filled Cells:**
If the cell at `pos` is already filled (i.e., non-zero in our representation), there’s nothing to do at this position. Recursively call `solve_sudoku(sudoku, next_pos(pos))` to continue solving from the next cell.
3. **Try Possible Digits:**
If the cell at pos is empty, first obtain the list of digits that can validly be placed there by calling `get_available_digits(sudoku, pos)`.
Then:
    - For each candidate digit `d` in that list:
        1. Place `d` in the cell at pos.
        2. Recursively call `solve_sudoku(sudoku, next_pos(pos))`.
        3. If this recursive call returns a valid solution, propagate that solution upward (return it immediately).
    - If none of the candidate digits lead to a valid solution, reset the cell to empty and backtrack.
---
This recursive exploration systematically tries every valid possibility, backing up whenever it reaches a dead end. Eventually, it either returns a complete, consistent Sudoku grid or determines that no solution exists.